In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import time, datetime, timedelta
import random

# Parameters
hours_open = 11  # Assuming 8 hours of operation per day
average_customers_per_day = 100  # Average customers per day

def generatePoissonProcessArrivals(hours_open, average_customers_per_day):
    lambda_val = average_customers_per_day / hours_open
    total_customers = np.random.poisson(average_customers_per_day)
    arrival_times = np.cumsum(np.random.exponential(1 / lambda_val, total_customers) * 60)
    return arrival_times


arrival_times = generatePoissonProcessArrivals(hours_open, average_customers_per_day)

def generateIncomingDemandTimestamps(arrival_times):
    start_time = datetime.strptime('08:00', '%H:%M')
    arrival_datetimes = [start_time + timedelta(minutes=int(t)) for t in arrival_times]
    arrival_minutes = [(dt - start_time).seconds // 60 for dt in arrival_datetimes]
    # Print arrival timestamps
    print(len(arrival_minutes),"customers arrived!")
    arrival_timestamps = [dt.strftime('%H:%M:%S') for dt in arrival_datetimes]
    
    return arrival_timestamps



arrival_timestamps = generateIncomingDemandTimestamps(arrival_times)

92 customers arrived!


In [19]:
class Pharmacy:
    def __init__(self, pharmacy_name, initial_balance, initial_inventory, old_inventory, unit_cost = 10, 
                 unit_revenue = 20, holding_cost = 2, outdating_cost = 5):
        self.balance = initial_balance
        self.name = pharmacy_name
        self.inventory = {'new': initial_inventory, 'old': old_inventory}
        self.revenue_per_unit = unit_revenue
        self.cost_per_unit = unit_cost
        self.holding_cost = holding_cost
        self.outdating_cost = outdating_cost
    
    def replenish(self, amount):
        self.inventory['new'] += amount
        self.balance -= amount * self.cost_per_unit  # Assume each replenished item costs 10 unit of currency
    
    def process_day_end(self):
        # Apply holding cost for all items
        total_inventory = self.inventory['new'] + self.inventory['old']
        self.balance -= self.holding_cost * total_inventory
        
        # Apply outdating cost for old items that perish
        self.balance -= self.outdating_cost * self.inventory['old']

        # 'old' items perish at the end of the day
        self.inventory['old'] = 0
        # 'new' items become 'old' at the end of the day
        self.inventory['old'] = self.inventory['new']
        self.inventory['new'] = 0
    
    # FIFO 
    def fulfill_demand(self):
        if self.inventory['old'] > 0:
            self.inventory['old'] -= 1
            self.balance += self.revenue_per_unit
            return True
        elif self.inventory['new'] > 0:
            self.inventory['new'] -= 1
            self.balance += self.revenue_per_unit
            return True
        else:
            # Implicit lost sales cost (not explicit) = - (revenue - cost)
            lost_sales_cost = -(self.revenue_per_unit - self.cost_per_unit)
            self.balance += lost_sales_cost
            return False
            print("No inventory to fulfill demand")
           
    def get_inventory_level(self):
        return self.inventory['new'] + self.inventory['old']
    
    def get_shelf_life_value(self):
        return 2 * self.inventory['new'] + self.inventory['old']
    
    def get_name(self):
        return "Pharmacy " + self.name
    
    def __str__(self):
        return f"Balance: {self.balance}, Inventory: {self.inventory}"
    
    def __repr__(self):
        return self.get_name()    

In [20]:
def convert_to_time(timestamp_str):
    return datetime.strptime(timestamp_str, "%H:%M:%S").time()

def isValidArrival(arrival_time):
    if(convert_to_time(arrival_time) >= opening_time and convert_to_time(arrival_time) <= closing_time):
        return True
    else:
        return False
    
def g(x):
    if x < 0:
        return -1
    elif x == 0:
        return 0
    else:
        return 1
    
def get_winner(pharmacy1, pharmacy2):
    if(pharmacy1.balance > pharmacy2.balance):
        return pharmacy1.get_name()
    elif(pharmacy1.balance < pharmacy2.balance):
        return pharmacy2.get_name()
    else:
        return "tie"
    
def fulfillIncomingDemand(chosen_pharmacy, other_pharmacy):
    if chosen_pharmacy.fulfill_demand():
        return None
    else:
        other_pharmacy.fulfill_demand()
    
def calculate_fulfillment_metric(I1, I2, V1, V2, w1 = 1, w2 = 1):
    if max(I1, I2) == 0:
        normalized_inventory_difference = 0
    else:
        normalized_inventory_difference = (g(I1 - I2) * abs(I1 - I2)) / max(I1, I2)
    return w1 * normalized_inventory_difference + w2 * (V1 - V2)

def simulate_day(arrival_timestamps, initial_balance, replenish_amount):
    invalid_arrivals = []

    pharmacy1 = Pharmacy("1", initial_balance, 20, 10)
    pharmacy2 = Pharmacy("2", initial_balance2, 10, 10)
    
    pharmacy1.replenish(replenish_amount)
    pharmacy2.replenish(replenish_amount2)
    
    print("Replenishment is done, Inventory levels at the beginning of day:")
    print(f"Pharmacy 1: {pharmacy1}")
    print(f"Pharmacy 2: {pharmacy2}")
    
    for timestamp in arrival_timestamps:
        if(isValidArrival(timestamp)):
            I1 = pharmacy1.get_inventory_level()
            I2 = pharmacy2.get_inventory_level()
            V1 = pharmacy1.get_shelf_life_value()
            V2 = pharmacy2.get_shelf_life_value()

            fulfillment_metric = calculate_fulfillment_metric(I1, I2, V1, V2)
            print(fulfillment_metric)

            if fulfillment_metric > 0:
                chosen_pharmacy = pharmacy1
                other_pharmacy = pharmacy1 if chosen_pharmacy == pharmacy2 else pharmacy2
                fulfillIncomingDemand(chosen_pharmacy, other_pharmacy)
            elif fulfillment_metric < 0:
                chosen_pharmacy = pharmacy2
                other_pharmacy = pharmacy1 if chosen_pharmacy == pharmacy2 else pharmacy2
                fulfillIncomingDemand(chosen_pharmacy, other_pharmacy)
            else:
                chosen_pharmacy = random.choice([pharmacy1, pharmacy2])
                other_pharmacy = pharmacy1 if chosen_pharmacy == pharmacy2 else pharmacy2
                fulfillIncomingDemand(chosen_pharmacy, other_pharmacy)
                
            print(f"At timestamp {timestamp}, chosen pharmacy: {chosen_pharmacy.get_name()}")
            print(f"Pharmacy 1: {pharmacy1}")
            print(f"Pharmacy 2: {pharmacy2}")
        else:
            invalid_arrivals.append(timestamp)

        
    # Process end of day changes
    pharmacy1.process_day_end()
    pharmacy2.process_day_end()
    
    print("End of day:")
    print(f"Pharmacy 1: {pharmacy1}")
    print(f"Pharmacy 2: {pharmacy2}")
    print(f"# of unresponsive customer: {len(invalid_arrivals)}")
    print("- - - - - - - - - - - - -")
    print(f"Winner: {get_winner(pharmacy1, pharmacy2)}")


In [21]:
initial_balance = 100
initial_balance2 = 100
replenish_amount = 5
replenish_amount2 = 5
opening_time = time(8, 0, 0)
closing_time = time(18, 0, 0)


simulate_day(arrival_timestamps, initial_balance, replenish_amount)

Replenishment is done, Inventory levels at the beginning of day:
Pharmacy 1: Balance: 50, Inventory: {'new': 25, 'old': 10}
Pharmacy 2: Balance: 50, Inventory: {'new': 15, 'old': 10}
20.285714285714285
At timestamp 08:15:00, chosen pharmacy: Pharmacy 1
Pharmacy 1: Balance: 70, Inventory: {'new': 25, 'old': 9}
Pharmacy 2: Balance: 50, Inventory: {'new': 15, 'old': 10}
19.264705882352942
At timestamp 08:31:00, chosen pharmacy: Pharmacy 1
Pharmacy 1: Balance: 90, Inventory: {'new': 25, 'old': 8}
Pharmacy 2: Balance: 50, Inventory: {'new': 15, 'old': 10}
18.242424242424242
At timestamp 08:44:00, chosen pharmacy: Pharmacy 1
Pharmacy 1: Balance: 110, Inventory: {'new': 25, 'old': 7}
Pharmacy 2: Balance: 50, Inventory: {'new': 15, 'old': 10}
17.21875
At timestamp 08:48:00, chosen pharmacy: Pharmacy 1
Pharmacy 1: Balance: 130, Inventory: {'new': 25, 'old': 6}
Pharmacy 2: Balance: 50, Inventory: {'new': 15, 'old': 10}
16.193548387096776
At timestamp 08:55:00, chosen pharmacy: Pharmacy 1
Pharmac